### fin_data_cleaning

# Data cleaning and setting up of dataframes to plot

This notebook, sets out the data cleaning and forming of dfs to feed into `~/Jupyter/our_finances/random_data/random_dashboard/fin_dashboard.ipynb`

This notebook is a much simplified version of my actaul notebook for my real financial dashboard, because this notebook uses randomized data, rather than our real financial data, for obvious reasons. 

Key sections of notebook:
- [Datetime and index set](#1)
- [Setting up dfs for visualization](#2)
- [Credit card monthly mean](#3)
- [Checking account - cumsum and total spending/savings](#4)
- [Verifying monthly salary](#5)
- [Key categories analysis](#6)
- [Amazon](#7)
- [Groceries](#8)
- [Costco](#9)
- [Annual comparison of expense categories](#10)

# Randomizing process

To randomized our financial data, I did use our own financial data as a base. I randomized the data following the categories that we have. I used summary statistics for our actual financial data, to inform a truncated normalized function to randomize the data. In some instances, this method gave a satisfactory result, for other categories, I had to do a work around, so that the randomizing function provided figures that were real world believable. 

The assumption of a normal distribution of each category's transactions is incorrect, clearly captured in the plotting of the real and randomized data histograms. However, I determined it to be sufficient for the purposes of publishing the dashboard, as I did not want the randomization to be too similar to our actual transaction data. 

I did not randomize every category of data, instead I focused on the ones that represent either high number of transactions or high total expenditure. 

The two key functions I used to produced the data included: generating random set of dates and generating the random values of the transactions. 

### Generating random dates

`start = pd.to_datetime('2018-01-01')
end = pd.to_datetime('2021-03-31')

time_cat_ch = {}

for c in cat_ch:
    num = int(checking_values.loc[c]['count_nonzero'])
    time_cat_ch[c] = random_dates(start, end, num)`
    
### Generating random transaction values
    
`ts_series_cat_ch_trun = {}

for c in cat_ch:
    #print(c)
    
    mean = checking_values.loc[c]['mean']
    
    std = checking_values.loc[c]['std']
    
    lower = checking_values.loc[c]['min']
    
    upper = checking_values.loc[c]['max']
    
    total = int(checking_values.loc[c]['count_nonzero'])
    
    a, b = (lower - mean) / std, (upper - mean) / std
    
    #print(a, b)
    rand_data = truncnorm.rvs(a, b, size=total, loc=mean, scale=std)

    ts = time_cat_ch[c]
    ts_series_cat_ch_trun[c] = pd.Series(data=rand_data, index=ts, name='random_amt')`



In [1]:
# importing files

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from matplotlib.dates import DateFormatter

credit = pd.read_csv('/Users/fiona/pandas_tutorials/our_financials/random/credit_may_21.csv')
checking = pd.read_csv('/Users/fiona/pandas_tutorials/our_financials/random/checking_may_21.csv')

# for checking import
# print(credit.shape, checking.shape)


In [2]:
credit.head()

,date,category,rand_amt,payee
0,2018-01-01 23:00:01,Groceries,34.287725,0
1,2018-01-02 17:48:04,Groceries,59.720494,0
2,2018-01-03 00:33:06,Groceries,11.418539,0
3,2018-01-03 17:57:02,Groceries,101.858860,0
4,2018-01-06 02:48:40,Groceries,21.880803,0


<a id='1'></a>
# Datetime and index set

## Credit account

In [3]:
# creating datetime column
credit['datetime'] = pd.to_datetime(credit['date'], format= '%Y-%m-%d %H:%M:%S')
credit = credit.set_index('datetime')
credit = credit.drop(['date'], axis=1)
credit.head()

,category,rand_amt,payee
datetime,,,
2018-01-01 23:00:01,Groceries,34.287725,0
2018-01-02 17:48:04,Groceries,59.720494,0
2018-01-03 00:33:06,Groceries,11.418539,0
2018-01-03 17:57:02,Groceries,101.858860,0
2018-01-06 02:48:40,Groceries,21.880803,0


## Checking account

In [4]:
checking.head()

,date,category,rand_amt,payee
0,2019-09-12 19:21:52,Groceries,-507.566831,Costco
1,2020-12-16 06:48:09,Groceries,-392.264716,Costco
2,2020-05-07 18:32:21,Groceries,-356.360551,Costco
3,2018-10-09 01:17:28,Groceries,-350.176868,Costco
4,2019-10-23 20:47:59,Groceries,-317.838109,Costco


In [5]:
checking['datetime'] = pd.to_datetime(checking['date'], format= '%Y-%m-%d %H:%M:%S')
checking = checking.set_index('datetime')
checking = checking.drop(['date'], axis=1)
checking.sort_index().tail()

,category,rand_amt,payee
datetime,,,
2021-03-26 00:56:41,Entertainment,-22.115084,0
2021-03-26 09:13:49,Dining:Coffee,-15.139652,0
2021-03-27 09:11:51,Groceries,-124.631286,0
2021-03-29 22:59:07,Groceries,-227.566413,0
2021-03-31 00:00:00,Credit Card,-4151.655751,0


In [6]:
# checking['rand_amt'] = checking['rand_amt']*-1
checking.head()

,category,rand_amt,payee
datetime,,,
2019-09-12 19:21:52,Groceries,-507.566831,Costco
2020-12-16 06:48:09,Groceries,-392.264716,Costco
2020-05-07 18:32:21,Groceries,-356.360551,Costco
2018-10-09 01:17:28,Groceries,-350.176868,Costco
2019-10-23 20:47:59,Groceries,-317.838109,Costco


<a id='2'></a>
# Setting up dfs for visualization

Using `checking` and `credit` dfs

Original non randomized dataset required that clean the payee column, to remove extra text, to drop a number of unuseful columns. 

This section, I prepare individual dataframes to feed into my plot functions on my dashboard. 

## Credit card monthly expenditure

In [7]:
# using payments from checking account to credit card to track credit card balance monthly
credit_by_month = checking[checking['category'] == 'Credit Card']
credit_by_month = credit_by_month.sort_index()
credit_by_month['rand_amt'] = credit_by_month['rand_amt']*-1 # changing sign of expenditure just for plotting
credit_month = credit_by_month.drop(columns=['category', 'payee'])
# credit_month.sort_values(by='rand_amt')

# confirming output
# print(credit_month.tail(10), credit_month.count())

## ** Graph of monthly balance of credit card is included in dashboard

<a id='3'></a>
# Credit card monthly mean

In [8]:
## Used for graphing means of balance per month per year 
credit_year_mean = round(credit_month.resample('Y').mean(),0)
credit_year_mean

,rand_amt
datetime,
2018-12-31,4822.0
2019-12-31,3876.0
2020-12-31,4716.0
2021-12-31,4649.0


## Need to update here when beginning new year

In [9]:
# creating df to display means of monthly credit card spend and comparison with latest month
credit_year_mean.loc['current_month'] = round(credit_month.iloc[-1]['rand_amt'],0)
credit_year_mean['colors'] = ['green', 'green', 'green', 'green','orange']
credit_year_mean['period'] = [2018, 2019, 2020, 2021, 'current']
credit_year_mean = credit_year_mean.set_index('period')
credit_year_mean

,rand_amt,colors
period,,
2018,4822.0,green
2019,3876.0,green
2020,4716.0,green
2021,4649.0,green
current,4152.0,orange


## ** Plot bar graph of mean monthly repayments against current month.

In [10]:
# checking that this pull out the correct figure
credit_year_mean.loc[2020].rand_amt

4716.0

In [11]:
def credit_card_performance(df):
    if df['rand_amt'].iloc[3]<df['rand_amt'].iloc[2]:
        print('LOW CREDIT CARD: Below mean credit card total last month')
    else:
        print('HIGH CREDIT CARD: Above mean credit card total last month')

<a id='4'></a>
# Checking account

In [12]:
checking.head()

,category,rand_amt,payee
datetime,,,
2019-09-12 19:21:52,Groceries,-507.566831,Costco
2020-12-16 06:48:09,Groceries,-392.264716,Costco
2020-05-07 18:32:21,Groceries,-356.360551,Costco
2018-10-09 01:17:28,Groceries,-350.176868,Costco
2019-10-23 20:47:59,Groceries,-317.838109,Costco


## Monthly spending/savings 

In [13]:
# calculating monthly total of spending/savings
monthly_checking = round(checking.resample('M').sum(),2)
# print(monthly_checking.head(2))

# adding in a col of colors to use when plotting
monthly_checking['colors'] = ['orange' if x<0 else 'blue' for x in monthly_checking.rand_amt]
monthly_checking.head(10)

,rand_amt,colors
datetime,,
2018-01-31,315.18,blue
2018-02-28,7632.53,blue
2018-03-31,3839.96,blue
2018-04-30,7279.55,blue
2018-05-31,-1443.73,orange
2018-06-30,7028.07,blue
2018-07-31,-2045.74,orange
2018-08-31,-1947.65,orange
2018-09-30,7696.42,blue


## Balance of checking acc each month

In [14]:
june = checking.loc['2019-06-01':'2019-06-30']
june

,category,rand_amt,payee
datetime,,,
2019-06-25 05:07:00,Groceries,-291.659073,Costco
2019-06-08 21:06:25,Groceries,-199.260561,0
2019-06-17 19:37:19,Groceries,-179.578617,0
2019-06-29 00:11:46,Groceries,-113.295657,0
2019-06-01 03:51:40,Groceries,-96.353283,0
2019-06-03 13:52:56,Groceries,-75.347292,0
2019-06-05 19:04:44,Babysitter,-85.780255,0
2019-06-17 04:37:48,Babysitter,-121.016793,0
2019-06-22 00:32:16,Babysitter,-199.118005,0


In [15]:
# calculating the balance in checking account over time
monthly_cumsum = monthly_checking['rand_amt'].cumsum()

# make it df to access values in df more easily
monthly_cumsum_df = pd.DataFrame(monthly_cumsum)
monthly_cumsum_df.head(3)

,rand_amt
datetime,
2018-01-31,315.18
2018-02-28,7947.71
2018-03-31,11787.67


## ** Plotting graph of monthly balance of checking account and total savings/spending each month

<a id='5'></a>
# Verifying monthly salary

In [16]:
salary = checking[checking.category == 'Salary']
# print(len(salary))

monthly_salary = round(salary.resample('M').sum(),2)
# print(monthly_salary.head())

monthly_salary['year'] = monthly_salary.index.year
monthly_salary['month'] = monthly_salary.index.month

monthly_salary.head()

,rand_amt,year,month
datetime,,,
2018-01-31,15653.04,2018,1
2018-02-28,16182.32,2018,2
2018-03-31,16283.66,2018,3
2018-04-30,15227.98,2018,4
2018-05-31,15686.89,2018,5


In [17]:
salary_year = pd.pivot_table(monthly_salary, index='month', columns='year', values='rand_amt')
salary_year.head(2)

year,2018,2019,2020,2021
month,,,,
1,15653.04,16564.62,16844.61,13974.38
2,16182.32,14461.74,15717.94,16149.92


## ** Plotting: monthly salary for each month by year

In [18]:
yearly_salary = salary.resample('Y').sum()
yearly_salary

,rand_amt
datetime,
2018-12-31,184597.988277
2019-12-31,182252.847900
2020-12-31,177859.885313
2021-12-31,44805.105035


<a id='6'></a>
# Key spending categories

These categories are the ones that we have the most control over and can be quite large.

Includes:
- Amazon
- Groceries
- Costco

## Combining checking and credit data
Need to combine for other categories because expenditure happens from both checking and credit accounts.

In [19]:
checking.head()

,category,rand_amt,payee
datetime,,,
2019-09-12 19:21:52,Groceries,-507.566831,Costco
2020-12-16 06:48:09,Groceries,-392.264716,Costco
2020-05-07 18:32:21,Groceries,-356.360551,Costco
2018-10-09 01:17:28,Groceries,-350.176868,Costco
2019-10-23 20:47:59,Groceries,-317.838109,Costco


In [20]:
checking_to_combine = checking.copy()
checking_to_combine['rand_amt'] = checking_to_combine['rand_amt']*-1 # used when changing sign of checking account
combined = pd.concat([credit, checking_to_combine])
combined = combined.sort_index()
combined.head()

,category,rand_amt,payee
datetime,,,
2018-01-01 00:00:13,Salary,-15653.042027,0
2018-01-01 10:04:30,Home:Baby,47.353480,amazon
2018-01-01 23:00:01,Groceries,34.287725,0
2018-01-01 23:54:06,Entertainment,35.376280,0
2018-01-02 06:08:47,Transportation,55.538356,0


In [21]:
combined.category.value_counts().head()

Groceries         774
Transportation    426
Dining:Coffee     367
Medical:Doctor    221
Entertainment     170
Name: category, dtype: int64

<a id='7'></a>
# Amazon

Only use credit to pay for amazon items, so only have to use credit card.

Amazon has many different payee descriptions, so in the original code, had to collect all various permutations. 

`amazon = credit_a[credit_a.payee_c.str.contains('AMAZON|AMZN', na=False, case=False)].copy()`

This does not include any audible transactions. 

In [22]:
credit.head(1)

,category,rand_amt,payee
datetime,,,
2018-01-01 23:00:01,Groceries,34.287725,0


In [23]:
amazon = credit[credit['payee'] =='amazon'].copy()

# cleaning df to include just columns I want datetime and amt_adj
amazon = amazon.drop(columns=['category', 'payee'])
# print(amazon.head())

# attempt to add monthly transactions
monthly_amazon = round(amazon.resample('M').sum(),2)
monthly_amazon.head()

,rand_amt
datetime,
2018-01-31,358.07
2018-02-28,331.27
2018-03-31,154.83
2018-04-30,502.86
2018-05-31,142.13


## ** plotting: monthly_bar(monthly_amazon, 'Monthly expenditure with Amazon', monthly_amazon['amt_adj'])

<a id='8'></a>
# Groceries

In [24]:
# seperating out groceries expenditure
groceries = combined[combined.category == 'Groceries'].copy()
# print(groceries.head(20))

# resampling so I can plot monthly groceries expenditure
monthly_groceries = groceries.resample('M').sum()
monthly_groceries.head()

,rand_amt
datetime,
2018-01-31,1790.954622
2018-02-28,967.249675
2018-03-31,1635.018668
2018-04-30,1000.104081
2018-05-31,1267.995192


## ** Plotting: monthly_bar(monthly_groceries_expenditure, 'Monthly groceries expenditure', monthly_groceries_expenditure['amount'])

In [25]:
# Yearly groceries expenditure
yearly_groceries = groceries.resample('Y').sum()
yearly_groceries

,rand_amt
datetime,
2018-12-31,15582.315739
2019-12-31,17636.449139
2020-12-31,17953.750893
2021-12-31,4348.934941


<a id='9'></a>
# Costco 
Trying to understand what is driving my groceries expenditure

In [26]:
costco = groceries[groceries['payee'] == 'Costco']
monthly_costco = costco.resample('M').sum()
monthly_costco.head()

,rand_amt
datetime,
2018-03-31,259.776335
2018-04-30,0.000000
2018-05-31,0.000000
2018-06-30,0.000000
2018-07-31,0.000000


## ** Plotting: monthly_bar(costco_monthly, 'Monthly Costco expenditure', costco_monthly['amount'])

In [27]:
yearly_costco = costco.resample('Y').sum()
yearly_costco

,rand_amt
datetime,
2018-12-31,885.570271
2019-12-31,2686.337849
2020-12-31,3789.351296
2021-12-31,271.215405


<a id='10'></a>
# Annual comparisons of expense categories

Interested in broader range of expense categories for total annual spend comparison

In [28]:
categories_spending = round(combined[['rand_amt', 'category']].groupby('category').sum(),0)
categories_spending = categories_spending.sort_values(by='rand_amt')
categories_spending.head()

,rand_amt
category,
Salary,-589516.0
Tax Refund,-4901.0
Interest Inc,-566.0
Utilities:Gas,1541.0
Utilities:Internet,2194.0


## Simplifying some categories
To do some annual analysis want to merge some categories, such as:
- merge auto
- merge holidays
- merge medical
- but want to keep dining and home separate

In [29]:
combined_cat = combined.copy()

In [30]:
# keeping dining and home separate
combined_cat['category'] = combined_cat['category'].str.replace('Dining:', '').str.replace('Home:', '')

# combining other categories, medical, auto, holidays
combined_cat['category'] = combined_cat['category'].str.replace(':\w+', '')

# all years spending
combined_cat_spending = round(combined_cat[['rand_amt', 'category']].groupby('category').sum(),0)

# confirming results
combined_cat_spending.sort_values('rand_amt').head(2)

,rand_amt
category,
Salary,-589516.0
Tax Refund,-4901.0


## Annual df for yearly analysis

In [31]:
combined_cat.head(3)

,category,rand_amt,payee
datetime,,,
2018-01-01 00:00:13,Salary,-15653.042027,0
2018-01-01 10:04:30,Baby,47.353480,amazon
2018-01-01 23:00:01,Groceries,34.287725,0


In [32]:
# adding year so can use it in pivot table
combined_cat['year'] = combined_cat.index.year

# pivot table to set up data to plot for annual analysis
categories_year = combined_cat.pivot_table(index='category', columns='year', aggfunc='sum')
categories_year.head()

# drop the multiindex so can access columns more simply
categories_year.columns = categories_year.columns.droplevel(0)
categories_year.head()

year,2018,2019,2020,2021
category,,,,
Auto,2217.786423,2167.073261,17193.033698,216.921869
Baby,3389.048999,2951.920576,2086.571671,1162.247219
Babysitter,9961.435450,7739.417224,7377.221921,517.105319
Clothing,1703.590014,1168.492286,776.706853,142.060435
Coffee,925.667580,867.081611,973.931637,321.109658


In [33]:
# Categories of interest to plot differences between years
# want also amazon 
misc_categories = ['Gift Given', 'Medical']
big_categories = ['Holidays', 'Groceries']
baby_categories = ['Babysitter', 'Baby']
dining_entertainment = ['Coffee','Meals', 'Entertainment','Drink']
auto_transport = ['Transportation', 'Auto']
savings_vanguard = ['Savings', 'Vanguard']